In [ ]:
merged_tickers = [
    "ITOT", "IYY", "SCHB", "SPTM", "OEF", "XLG", "IVV", "SPY", "SPLG", "VOO",   # S&P 500 & Total Market ETFs
    "DIA", "SCHX", "IJH", "MDY", "SPMD", "IVOO", "SCHM", "VXF", "IJR", "SPSM",  # Other large-cap ETFs
    "VIOO", "SCHA", "FDM", "SNPE", "EFIV", "RSPE", "XVV", "MIDE", "XJH", "SMLE",  # Broad exposure funds
    "ESIX", "XJR", "SNPG", "SNPV", "SNPD", "SPYX", "ICLN", "CGW", "CATH", "GRNB", # Green energy & other sector funds
    "KRBN", "KCCA", "KEUA", "SCHD", "DVY", "ISPY", "NOBL", "DIVG", "SPYD", "SPHD", # Dividend funds
    "KNGZ", "SDY", "REGL", "XSHD", "SPDG", "TDV", "VTEB", "TAXF", "AVMU", "DFNM", # Bond and fixed-income ETFs
    "IBMM", "IBMN", "IBMO", "IBMP", "IBMQ", "IBMR", "LQD", "SLQD", "LQDB", "HYG",  # Corporate bonds
    "SHYG", "TDTF", "TDTT", "XLC", "XLY", "XLE", "XLF", "XLV", "XLI", "XLB", "XLRE", # Sector ETFs
    "XLK", "XLU", "PSCD", "PSCC", "PSCE", "PSCF", "PSCH", "PSCI", "PSCT", "PSCM", # Small and mid-cap sector funds
    "PSCU", "KOMP", "HAIL", "FITE", "SIMS", "ROKT", "CNRG", "MOON", "CTEX", "MAKX", # Thematic ETFs
    "VXX", "VIXY", "VIXM", "FJAN", "DJAN", "FFEB", "DFEB", "FMAR", "DMAR", "FAPR", # Various ETFs
    "DAPR", "FMAY", "DMAY", "FJUN", "DJUN", "FJUL", "DJUL", "FAUG", "DAUG", "FSEP", # Funds for seasonal exposure
    "DSEP", "FOCT", "DOCT", "FNOV", "DNOV", "FDEC", "DDEC"  # Seasonal funds
]


In [57]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers categorized by caps
categories = {
    "Large-Cap": [
        "ITOT", "IYY", "SCHB", "SPTM", "IVV", "SPY", "VOO", "SCHX", "DIA", "VTI", "VV"
    ],
    "Mid-Cap": [
        "MDY", "IJH", "VO", "SCHM", "SPMD"
    ],
    "Small-Cap": [
        "IJR", "VB", "SCHA", "VIOO", "SPSM"
    ],
    "Micro-Cap": [
        "BND", "SCHA", "SCHB", "SCHM", "SCHX", "SPMD", "SPSM", "SPTM", "VB", "VIOO", 
        "VTEB", "VV", "XLC", "XLRE"
    ]
}

# Define the date range
start_date = "2021-01-01"
end_date = "2024-11-29"

# Fetch historical price and volume data for all tickers
all_tickers = [ticker for sublist in categories.values() for ticker in sublist]
fund_data = {}
print("Fetching historical data for all tickers...")
for ticker in all_tickers + ["SPY"]:  # Add a benchmark (e.g., SPY for S&P 500)
    try:
        print(f"Fetching data for {ticker}...")
        data = yf.download(ticker, start=start_date, end=end_date)[['Close', 'Volume']]
        if not data.empty:
            data['MarketCap'] = data['Close'] * data['Volume']  # Calculate Market Cap
            fund_data[ticker] = data
        else:
            print(f"No historical data available for {ticker}.")
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

# Combine all data into a single DataFrame with an additional 'Ticker' column
all_data = []
for ticker, data in fund_data.items():
    data['Ticker'] = ticker
    data['Date'] = data.index  # Ensure 'Date' column is present
    all_data.append(data)
human_fund = pd.concat(all_data)

# Ensure 'Date' is in datetime format
human_fund['Date'] = pd.to_datetime(human_fund['Date'])

# Drop funds with insufficient data
human_fund = human_fund.groupby('Ticker').filter(lambda x: len(x) >= 252)  # At least one year of data

# Extract market benchmark returns (SPY)
benchmark_data = human_fund[human_fund['Ticker'] == "SPY"]
benchmark_returns = benchmark_data['Close'].pct_change().dropna()

# Function to calculate portfolio metrics
def calculate_portfolio_metrics(returns, weights, benchmark_returns):
    portfolio_returns = np.dot(returns, weights)
    metrics = {
        'Annualized Return (%)': (1 + np.mean(portfolio_returns)) ** 252 - 1,
        'Sharpe Ratio': (np.mean(portfolio_returns) / np.std(portfolio_returns) * np.sqrt(252)) if np.std(portfolio_returns) else np.nan,
        'Sortino Ratio': (np.mean(portfolio_returns) / np.std(portfolio_returns[portfolio_returns < 0]) * np.sqrt(252)) if np.std(portfolio_returns[portfolio_returns < 0]) else np.nan,
        'Standard Deviation (%)': np.std(portfolio_returns) * np.sqrt(252),
        'Downside Deviation (%)': np.std(portfolio_returns[portfolio_returns < 0]) * np.sqrt(252) if np.std(portfolio_returns[portfolio_returns < 0]) else np.nan,
        'Beta': (np.cov(portfolio_returns, benchmark_returns)[0, 1] / np.var(benchmark_returns)) if np.var(benchmark_returns) else np.nan
    }
    return pd.Series(metrics)

# Function to compute equal variance weight allocation
def compute_equal_variance_weights(returns):
    vols = returns.std()
    inv_vols = 1 / vols  # Inverse volatilities
    weights = inv_vols / inv_vols.sum()  # Normalize to sum to 1
    return weights

# Process each category
portfolio_metrics = []
optimized_weights = {}

for category, tickers_list in categories.items():
    print(f"Processing {category} funds...")
    category_data = human_fund[human_fund['Ticker'].isin(tickers_list)]

    if category_data.empty:
        print(f"No valid data for {category}. Skipping.")
        continue

    # Pivot returns for weight allocation
    returns = category_data.pivot(index='Date', columns='Ticker', values='Close').pct_change().dropna()
    if returns.empty or returns.shape[1] < 2:
        print(f"Not enough data for {category}.")
        continue

    # Compute equal variance weights for the category
    weights = compute_equal_variance_weights(returns)
    optimized_weights[category] = pd.Series(weights, index=returns.columns)

    # Align benchmark returns to match the portfolio returns date range
    aligned_benchmark_returns = benchmark_returns.loc[returns.index]

    # Calculate portfolio metrics
    metrics = calculate_portfolio_metrics(returns.values, weights, aligned_benchmark_returns)
    metrics['Category'] = category
    portfolio_metrics.append(metrics)

# Consolidate metrics into a single DataFrame
metrics_df = pd.DataFrame(portfolio_metrics)
metrics_df

Fetching historical data for all tickers...
Fetching data for ITOT...
[*********************100%%**********************]  1 of 1 completed
Fetching data for IYY...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHB...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPTM...
[*********************100%%**********************]  1 of 1 completed
Fetching data for IVV...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPY...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VOO...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHX...
[*********************100%%**********************]  1 of 1 completed
Fetching data for DIA...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VTI...
[*********************100%%**********************]  1 of 1 completed
Fetching dat

/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MarketCap'] = data['Close'] * data['Volume']  # Calculate Market Cap
/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MarketCap'] = data['Close'] * data['Volume']  # Calculate Market Cap
/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarnin

[*********************100%%**********************]  1 of 1 completed
Fetching data for VO...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHM...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPMD...
[*********************100%%**********************]  1 of 1 completed
Fetching data for IJR...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VB...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHA...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VIOO...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPSM...
[*********************100%%**********************]  1 of 1 completed
Fetching data for BND...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHA...
[*********************100%%**********************]  1 of

/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MarketCap'] = data['Close'] * data['Volume']  # Calculate Market Cap
/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MarketCap'] = data['Close'] * data['Volume']  # Calculate Market Cap
/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarnin

[*********************100%%**********************]  1 of 1 completed
Fetching data for SCHX...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPMD...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPSM...
[*********************100%%**********************]  1 of 1 completed
Fetching data for SPTM...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VB...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VIOO...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VTEB...
[*********************100%%**********************]  1 of 1 completed
Fetching data for VV...
[*********************100%%**********************]  1 of 1 completed
Fetching data for XLC...
[*********************100%%**********************]  1 of 1 completed
Fetching data for XLRE...
[*********************100%%**********************]  1 o

/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MarketCap'] = data['Close'] * data['Volume']  # Calculate Market Cap
/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['MarketCap'] = data['Close'] * data['Volume']  # Calculate Market Cap
/var/folders/0p/fxg5ckl91l97lc30f5yd7h840000gn/T/ipykernel_38522/1299746633.py:35: SettingWithCopyWarnin

,Annualized Return (%),Sharpe Ratio,Sortino Ratio,Standard Deviation (%),Downside Deviation (%),Beta,Category
0,0.139935,0.799994,1.130150,0.163758,0.115919,0.985820,Large-Cap
1,0.117820,0.582338,0.893867,0.191306,0.124632,1.034264,Mid-Cap
2,0.108494,0.478862,0.793476,0.215141,0.129837,1.066160,Small-Cap
3,0.058107,0.503435,0.752036,0.112205,0.075113,0.632951,Micro-Cap


In [58]:
# Display weights
print("\nOptimized Weights for Each Category:")
for category, weights in optimized_weights.items():
    print(f"\n{category}:")
    print(weights)


Optimized Weights for Each Category:

Large-Cap:
Ticker
DIA     0.105703
ITOT    0.087850
IVV     0.090730
IYY     0.089054
SCHB    0.088033
SCHX    0.088915
SPTM    0.090993
SPY     0.090581
VOO     0.091247
VTI     0.087988
VV      0.088905
dtype: float64

Mid-Cap:
Ticker
IJH     0.194623
MDY     0.194287
SCHM    0.199281
SPMD    0.195722
VO      0.216088
dtype: float64

Small-Cap:
Ticker
IJR     0.198568
SCHA    0.195495
SPSM    0.198690
VB      0.207997
VIOO    0.199251
dtype: float64

Micro-Cap:
Ticker
BND     0.150384
SCHA    0.043184
SCHB    0.056211
SCHM    0.049521
SCHX    0.056774
SPMD    0.048637
SPSM    0.043890
SPTM    0.058101
VB      0.045945
VIOO    0.044013
VTEB    0.253317
VV      0.056768
XLC     0.044457
XLRE    0.048797
dtype: float64
